In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['AF']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('atrial fibrillation', 5195),
 ('annulus fibrosus', 293),
 ('amniotic fluid', 127),
 ('aflatoxin', 101),
 ('auranofin', 91),
 ('autofluorescence', 64),
 ('aminofluorene', 52),
 ('anulus fibrosus', 39),
 ('antiferromagnetic', 37),
 ('antisecretory factor', 24),
 ('aspergillus fumigatus', 20),
 ('aqueous fraction', 20),
 ('arcuate fasciculus', 19),
 ('aminoflavone', 18),
 ('activity function', 16),
 ('amentoflavone', 15),
 ('ascofuranone', 12),
 ('activity fraction', 12),
 ('adventitial fibroblasts', 11),
 ('atrial fibrillation flutter', 11),
 ('af', 10),
 ('albiflorin', 10),
 ('acicular ferrite', 9),
 ('aortic flow', 9),
 ('aggressive fibromatosis', 9),
 ('antifouling', 9),
 ('asialofetuin', 9),
 ('atrial flutter', 8),
 ('acetone fraction', 8),
 ('anthocyanin fraction', 8),
 ('alcohol fed', 7),
 ('auto fluorescence', 7),
 ('allantoic fluid', 7),
 ('aggregation factor', 7),
 ('adipocyte fraction', 7),
 ('anhydro d fructose', 7),
 ('actin filament', 6),
 ('acne fulminans', 6),
 ('aerobi

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-18 19:33:48] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'cp fine': 'MESH:D052638',
  'f poae': 'ungrounded',
  'f prostanoid': 'CHEBI:CHEBI:26347',
  'f series prostanoid': 'CHEBI:CHEBI:26347',
  'fabry perot': 'ungrounded',
  'fabry pérot': 'ungrounded',
  'faecal protease': 'ungrounded',
  'faecalibacterium prausnitzii': 'MESH:D000070037',
  'false positive': 'MESH:D005189',
  'false positives': 'MESH:D005189',
  'family physicians': 'MESH:D010821',
  'family planning': 'MESH:D005193',
  'farnesyl phosphate': 'CHEBI:CHEBI:24018',
  'fast pathway': 'CHEBI:CHEBI:34922',
  'fat pad': 'MESH:D000273',
  'fat percentage': 'ungrounded',
  'fatty pancreas': 'ungrounded',
  'female protein': 'ungrounded',
  'fenpropimorph': 'CHEBI:CHEBI:50145',
  'fenugreek powder': 'ungrounded',
  'fermentation production': 'ungrounded',
  'fertility preservation': 'MESH:D059247',
  'fetal pancreas': 'ungrounded',
  'few polyhedra': 'ungrounded',
  'fgfb pacap': 'ungrounded',
  'fiber protein': 'MESH:D012596',
  'field potential': 'field_potential',
  'field po

In [6]:
from adeft.disambiguate import load_disambiguator
d = load_disambiguator('AF')
grounding_dict, names, pos_labels = d.grounding_dict, d.names, d.pos_labels
grounding_map = grounding_dict['AF']
result = [grounding_map, names, pos_labels]
result

[{'atrial fibrillation': 'MESH:D001281',
  'annulus fibrosus': 'MESH:D000070616',
  'amniotic fluid': 'MESH:D000653',
  'aflatoxin': 'CHEBI:CHEBI:22271',
  'auranofin': 'CHEBI:CHEBI:2922',
  'autofluorescence': 'MESH:D061848',
  'aminofluorene': 'PUBCHEM:22817',
  'anulus fibrosus': 'MESH:D000070616',
  'antiferromagnetic': 'antiferromagnetic',
  'antisecretory factor': 'MESH:C049628',
  'aspergillus fumigatus': 'MESH:D001232',
  'aqueous fraction': 'ungrounded',
  'arcuate fasciculus': 'ungrounded',
  'aminoflavone': 'MESH:C413760',
  'activity function': 'ungrounded',
  'amentoflavone': 'CHEBI:CHEBI:2631',
  'ascofuranone': 'MESH:C006640',
  'activity fraction': 'ungrounded',
  'adventitial fibroblasts': 'ungrounded',
  'atrial fibrillation flutter': 'MESH:D001281',
  'af': 'ungrounded',
  'albiflorin': 'CHEBI:CHEBI:132793',
  'acicular ferrite': 'ungrounded',
  'aortic flow': 'ungrounded',
  'aggressive fibromatosis': 'MESH:D018222',
  'antifouling': 'ungrounded',
  'asialofetuin': 

In [10]:
grounding_map, names, pos_labels = [{'atrial fibrillation': 'MESH:D001281',
  'annulus fibrosus': 'MESH:D000070616',
  'amniotic fluid': 'MESH:D000653',
  'aflatoxin': 'CHEBI:CHEBI:22271',
  'auranofin': 'CHEBI:CHEBI:2922',
  'autofluorescence': 'MESH:D061848',
  'aminofluorene': 'PUBCHEM:22817',
  'anulus fibrosus': 'MESH:D000070616',
  'antiferromagnetic': 'antiferromagnetic',
  'antisecretory factor': 'MESH:C049628',
  'aspergillus fumigatus': 'MESH:D001232',
  'aqueous fraction': 'ungrounded',
  'arcuate fasciculus': 'ungrounded',
  'aminoflavone': 'MESH:C413760',
  'activity function': 'ungrounded',
  'amentoflavone': 'CHEBI:CHEBI:2631',
  'ascofuranone': 'MESH:C006640',
  'activity fraction': 'ungrounded',
  'adventitial fibroblasts': 'ungrounded',
  'atrial fibrillation flutter': 'MESH:D001281',
  'af': 'ungrounded',
  'albiflorin': 'CHEBI:CHEBI:132793',
  'acicular ferrite': 'ungrounded',
  'aortic flow': 'ungrounded',
  'aggressive fibromatosis': 'MESH:D018222',
  'antifouling': 'ungrounded',
  'asialofetuin': 'MESH:C017029',
  'atrial flutter': 'MESH:D001282',
  'acetone fraction': 'ungrounded',
  'anthocyanin fraction': 'ungrounded',
  'alcohol fed': 'ungrounded',
  'auto fluorescence': 'MESH:D061848',
  'allantoic fluid': 'ungrounded',
  'aggregation factor': 'MESH:C013065',
  'adipocyte fraction': 'ungrounded',
  'anhydro d fructose': 'ungrounded',
  'actin filament': 'FPLX:Actin',
  'acne fulminans': 'ungrounded',
  'aerobic fitness': 'ungrounded',
  'ascitic fluid': 'MESH:D001202',
  'atrial fibrillation or flutter': 'MESH:D001281',
  'abdominal fat': 'MESH:D050153',
  'afferent facilitate': 'ungrounded',
  'arginyl fructose': 'ungrounded',
  'asthmatic fibroblasts': 'MESH:D005347',
  'activity factor': 'ungrounded',
  'f axysporum': 'ungrounded',
  'accelerated fraction': 'ungrounded',
  'attributable fraction': 'ungrounded',
  'a trial fibrillation': 'MESH:D001281',
  'access flap': 'ungrounded',
  'adaptive flies': 'ungrounded',
  'altered feedback': 'ungrounded',
  'arial fibrillation': 'ungrounded',
  'adherent fraction': 'ungrounded',
  'alkaloid fraction': 'ungrounded',
  'fak recombinant adenovirus': 'MESH:D000256',
  'annulus fibrosis': 'MESH:D005355',
  'allele frequency': 'MESH:D005787',
  'arrival formula': 'ungrounded',
  'adjacency filter': 'ungrounded',
  'alcohol fermentation': 'ungrounded',
  'adaptive function': 'ungrounded',
  'acifluorfen': 'CHEBI:CHEBI:73172',
  'amyloid fibrillation': 'ungrounded',
  'angiogenic factor': 'ungrounded',
  'aflatoxin b1': 'CHEBI:CHEBI:22271'},
 {'MESH:D001281': 'Atrial Fibrillation',
  'MESH:D050153': 'Abdominal Fat',
  'CHEBI:CHEBI:73172': 'acifluorfen',
  'FPLX:Actin': 'Actin',
  'CHEBI:CHEBI:22271': 'aflatoxin',
  'MESH:C013065': 'cell aggregation factors',
  'MESH:D018222': 'Fibromatosis, Aggressive',
  'CHEBI:CHEBI:132793': 'albiflorin',
  'MESH:D005787': 'Gene Frequency',
  'CHEBI:CHEBI:2631': 'amentoflavone',
  'MESH:C413760': 'aminoflavone',
  'PUBCHEM:22817': '1-Aminofluorene',
  'MESH:D000653': 'Amniotic Fluid',
  'MESH:D005355': 'Fibrosis',
  'MESH:D000070616': 'Annulus Fibrosus',
  'antiferromagnetic': 'antiferromagnetic',
  'MESH:C049628': 'antisecretory factor',
  'MESH:D001202': 'Ascitic Fluid',
  'MESH:C006640': 'ascofuranone',
  'MESH:C017029': 'asialofetuin',
  'MESH:D001232': 'Aspergillus fumigatus',
  'MESH:D005347': 'Fibroblasts',
  'MESH:D001282': 'Atrial Flutter',
  'CHEBI:CHEBI:2922': 'auranofin',
  'MESH:D061848': 'Optical Imaging',
  'MESH:D000256': 'Adenoviridae'},
 ['CHEBI:CHEBI:22271',
  'CHEBI:CHEBI:2922',
  'MESH:C049628',
  'MESH:D000070616',
  'MESH:D000653',
  'MESH:D001281',
  'MESH:D061848',
  'PUBCHEM:22817']]

In [11]:
excluded_longforms = ['af']

In [12]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = []

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entities = pos_labels + additional_entities
entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in entities}

In [17]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [18]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [19]:
intersection1

[('CHEBI:CHEBI:47344', 'CHEBI:CHEBI:47344', 0),
 ('CHEBI:CHEBI:47344', 'CHEBI:CHEBI:5130', 0),
 ('CHEBI:CHEBI:47344', 'HGNC:9600', 0),
 ('CHEBI:CHEBI:47344', 'MESH:D005189', 0),
 ('CHEBI:CHEBI:47344', 'MESH:D005193', 0),
 ('CHEBI:CHEBI:47344', 'MESH:D005454', 0),
 ('CHEBI:CHEBI:47344', 'MESH:D008164', 0),
 ('CHEBI:CHEBI:47344', 'MESH:D014760', 0),
 ('CHEBI:CHEBI:47344', 'NCIT:C32623', 0),
 ('CHEBI:CHEBI:5130', 'CHEBI:CHEBI:47344', 0),
 ('CHEBI:CHEBI:5130', 'CHEBI:CHEBI:5130', 1314),
 ('CHEBI:CHEBI:5130', 'HGNC:9600', 0),
 ('CHEBI:CHEBI:5130', 'MESH:D005189', 0),
 ('CHEBI:CHEBI:5130', 'MESH:D005193', 0),
 ('CHEBI:CHEBI:5130', 'MESH:D005454', 0),
 ('CHEBI:CHEBI:5130', 'MESH:D008164', 0),
 ('CHEBI:CHEBI:5130', 'MESH:D014760', 0),
 ('CHEBI:CHEBI:5130', 'NCIT:C32623', 0),
 ('HGNC:9600', 'CHEBI:CHEBI:47344', 0),
 ('HGNC:9600', 'CHEBI:CHEBI:5130', 0),
 ('HGNC:9600', 'HGNC:9600', 0),
 ('HGNC:9600', 'MESH:D005189', 0),
 ('HGNC:9600', 'MESH:D005193', 0),
 ('HGNC:9600', 'MESH:D005454', 0),
 ('HGN

In [75]:
intersection2

[('CHEBI:CHEBI:27812', 'CHEBI:CHEBI:27812', 0),
 ('CHEBI:CHEBI:27812', 'CHEBI:CHEBI:7035', 0),
 ('CHEBI:CHEBI:27812', 'FPLX:Mechanosensitive_ion_channels', 0),
 ('CHEBI:CHEBI:27812', 'HGNC:7321', 0),
 ('CHEBI:CHEBI:27812', 'MESH:D000080364', 0),
 ('CHEBI:CHEBI:27812', 'MESH:D000604', 0),
 ('CHEBI:CHEBI:27812', 'MESH:D059630', 0),
 ('MESH:D059630', 'CHEBI:CHEBI:27812', 0),
 ('MESH:D059630', 'CHEBI:CHEBI:7035', 0),
 ('MESH:D059630', 'FPLX:Mechanosensitive_ion_channels', 0),
 ('MESH:D059630', 'HGNC:7321', 0),
 ('MESH:D059630', 'MESH:D000080364', 0),
 ('MESH:D059630', 'MESH:D000604', 0),
 ('MESH:D059630', 'MESH:D059630', 2261),
 ('ungrounded', 'CHEBI:CHEBI:27812', 0),
 ('ungrounded', 'CHEBI:CHEBI:7035', 0),
 ('ungrounded', 'FPLX:Mechanosensitive_ion_channels', 0),
 ('ungrounded', 'HGNC:7321', 0),
 ('ungrounded', 'MESH:D000080364', 0),
 ('ungrounded', 'MESH:D000604', 0),
 ('ungrounded', 'MESH:D059630', 0),
 ('MESH:D000604', 'CHEBI:CHEBI:27812', 0),
 ('MESH:D000604', 'CHEBI:CHEBI:7035', 0),


[('HGNC:7321', ['Musculin', 'musculin'])]

In [91]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-22 04:53:59] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-22 04:56:38] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9735958052861943 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'MESH:D001232': 17,
  'MESH:D000653': 95,
  'CHEBI:CHEBI:2922': 72,
  'ungrounded': 139,
  'MESH:D001281': 3565,
  'CHEBI:CHEBI:22271': 86,
  'PUBCHEM:22817': 51,
  'MESH:C013065': 6,
  'MESH:C006640': 8,
  'MESH:D061848': 50,
  'MESH:C017029': 7,
  'CHEBI:CHEBI:73172': 3,
  'MESH:D000070616': 200,
  'MESH:C049628': 18,
  'MESH:D001202': 5,
  'MESH:D018222': 7,
  'MESH:D005347': 1,
  'antiferromagnetic': 31,
  'MESH:C413760': 14,
  'FPLX:Actin': 5,
  'MESH:D050153': 2,
  'CHEBI:CHEBI:2631': 13,
  'MESH:D001282': 7,
  'CHEBI:CHEBI:132793': 7,
  'MESH:D005355': 2,
  'MESH:D005787': 3,
  'MESH:D000256': 1},
 'f1': {'mean': 0.973596, 'std': 0.003303},
 'precision': {'mean': 0.966247, 'std': 0.00349},
 'recall': {'mean': 0.982355, 'std': 0.003633},
 'MESH:D000653': {'f1': {'mean': 0.853381, 'std': 0.042432},
  'pr': {'mean': 0.8, 'std': 0.061378},
  'rc': {'mean': 0.918977, 'std': 0.05525}},
 'MESH:C013065': {'f1': {'mean': 0.8, 'std': 0.4},
  'pr': {'mean': 0.8, 'st

In [19]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [20]:
disamb.dump(model_name, results_path)

In [21]:
print(disamb.info())

Disambiguation model for AF

Produces the disambiguations:
	1-Aminofluorene*	PUBCHEM:22817
	Abdominal Fat	MESH:D050153
	Actin	FPLX:Actin
	Adenoviridae	MESH:D000256
	Amniotic Fluid*	MESH:D000653
	Annulus Fibrosus*	MESH:D000070616
	Ascitic Fluid	MESH:D001202
	Aspergillus fumigatus	MESH:D001232
	Atrial Fibrillation*	MESH:D001281
	Atrial Flutter	MESH:D001282
	Fibroblasts	MESH:D005347
	Fibromatosis, Aggressive	MESH:D018222
	Fibrosis	MESH:D005355
	Gene Frequency	MESH:D005787
	Optical Imaging*	MESH:D061848
	acifluorfen	CHEBI:CHEBI:73172
	aflatoxin*	CHEBI:CHEBI:22271
	albiflorin	CHEBI:CHEBI:132793
	amentoflavone	CHEBI:CHEBI:2631
	aminoflavone	MESH:C413760
	antiferromagnetic	antiferromagnetic
	antisecretory factor*	MESH:C049628
	ascofuranone	MESH:C006640
	asialofetuin	MESH:C017029
	auranofin*	CHEBI:CHEBI:2922
	cell aggregation factors	MESH:C013065

Class level metrics:
--------------------
Grounding               	Count	F1     
     Atrial Fibrillation*	3565	0.98558
        Annulus Fibrosus*	 2

In [22]:
model_to_s3(disamb)